# Step 4 & 5: Predict using GRU and DNN models, calibrate, and get the final crop growth result


*   Use GRU model to find the most similar input from GRU-generated data

*   Use DNN model to predict the 24-hour crop growth from the closest environment parameters found within GRU generation in Step 3

*   Calibrate the output data from DNN to day input of Step 1. Calculate the difference rate, and multiply the rate to the value of each hour value.

*   Get the final crop growth result of the specified hour from the calibrated result (Select the hour from step 1). Error calculation.

# Libraries and settings

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import sklearn
from sklearn import metrics
import tensorflow as tf
import seaborn as sns
import os
from google.colab import drive

In [3]:
print(os.getcwd())
drive.mount('/content/drive')

/content
Mounted at /content/drive


# Load models and data

In [4]:
# load the saved models
model_GRU = tf.keras.models.load_model('/content/drive/MyDrive/INF_RP_CropModel/GRU_model_yuyanyang.keras')
model_GRU.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 9, 256)            204288    
                                                                 
 gru_1 (GRU)                 (None, 9, 128)            148224    
                                                                 
 gru_2 (GRU)                 (None, 9, 64)             37248     
                                                                 
 gru_3 (GRU)                 (None, 16)                3936      
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
Total params: 394104 (1.50 MB)
Trainable params: 394104 

In [5]:
model_DNN = tf.keras.models.load_model('/content/drive/MyDrive/INF_RP_CropModel/DNN_model_yuyanyang.keras')
model_DNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 27)                108       
                                                                 
 dense_1 (Dense)             (None, 50)                1400      
                                                                 
 dense_2 (Dense)             (None, 40)                2040      
                                                                 
 dense_3 (Dense)             (None, 35)                1435      
                                                                 
 dense_4 (Dense)             (None, 30)                1080      
                                                                 
 dense_5 (Dense)             (None, 24)                744       
                                                                 
Total params: 6807 (26.59 KB)
Trainable params: 6807 (26

In [6]:
# Load and pre-process data

# DNDC train data
df = pd.read_csv('/content/drive/MyDrive/INF_RP_CropModel/data_tomato_95cases_train.csv', header=0)
df = df.drop(columns="Unnamed: 0")
df = df.set_index("Day")
# DNDC test data
df_test = pd.read_csv('/content/drive/MyDrive/INF_RP_CropModel/data_tomato_test.csv', header=0)
df_test = df_test.drop(columns="Unnamed: 0")
df_test = df_test.set_index("Day")
# Greenlight augmented data
df_aug = pd.read_csv('/content/drive/MyDrive/INF_RP_CropModel/data_3to24_augmented.csv')
df_aug = df_aug.set_index("Day")

In [7]:
# normalize DNDC train data
scaler1 = sklearn.preprocessing.MinMaxScaler(feature_range=(0,1))
df_norm = df.copy()
for column in df_norm.columns.tolist():
    df_norm[column] = scaler1.fit_transform(df[column].values.reshape(-1,1))

In [8]:
# normalize Greenlight augmented data
scaler2 = sklearn.preprocessing.MinMaxScaler(feature_range=(0,1))
df_aug_norm = df_aug.copy()
for column in df_aug.columns.tolist():
    df_aug_norm[column] = scaler2.fit_transform(df_aug[column].values.reshape(-1,1))

In [9]:
seq_len = 10

# Step 4: Use GRU model to find the most similar input from GRU-generated data


In [254]:
# customize parameters
DAY = 120  # the day to predict crop parameters given environmental conditions (DAY>INCLUDE)
INCLUDE = 2  # to include how many previous days for searching for the similar
THRESHOLD = 0.4  # the acceptable deviation rate is within THRESHOLD

FORWARD = 90  # in the loop, to generate how many days for the next search
BACK = 10  # in the loop, to go back how many days for the next generation
ROUND_MAX = 90  # maximum rounds to generate new data using GRU model

In [255]:
# a function to get values out from nested lists or tuples
def flatten(seq):
    l = []
    for elt in seq:
        t = type(elt)
        if t is tuple or t is list:
            for elt2 in flatten(elt):
                l.append(elt2)
        else:
            l.append(elt)
    return l

In [256]:
# A function to loop over ref df, compare with test df, and find the most similar environmental parameter set in ref df.
def find_similar(test: pd.DataFrame, ref: pd.DataFrame):
    # select test data to compare: 4 of DAY and 8*INCLUDE
    list_test1 = test.iloc[DAY - test.index[0], :4].values.tolist()
    # print("Input:\n", test.iloc[DAY - test.index[0], :4])
    list_test2 = test.iloc[
        (DAY - test.index[0] - INCLUDE) : (DAY - test.index[0]), :
    ].values.tolist()
    list_test = flatten(list_test1 + list_test2)  # get values out from list of lists
    # check if any value=0 which would cause error in later calculation
    if 0 in list_test:
        raise Exception('Test values cannot be 0. Please check and adjust parameters.')
    else:
        devi_rates = []
        indexes = []
        # a loop to compare the target test data with crop data and stop when an acceptable deviation rate is found
        for i in range(len(ref)):
            if i > INCLUDE:
                list_ref1 = ref.iloc[i, :4].values.tolist()
                list_ref2 = ref.iloc[(i - INCLUDE) : i, :].values.tolist()
                list_ref = flatten(list_ref1 + list_ref2)

                # define deviation rate = all errors + Coefficient of variation of errors
                errs = [
                    (list_test[n] - list_ref[n]) / list_test[n]
                    for n in range(len(list_test))
                ]
                # cv = np.std(errs) / np.mean(errs)
                # devi_rate = sum(list(map(abs, errs))) + abs(cv)
                devi_rate = sum(list(map(abs, errs))) / len(errs)

                # put acceptable values and indexes into lists
                if abs(devi_rate) < THRESHOLD:
                    devi_rates.append(devi_rate)
                    indexes.append(i)

        # get the corresponding crop parameters where a minimal devi_rate is found (between test and crop data)
        if len(devi_rates) > 0:  # when any result exists
            # get the index of min devi_rate
            devi_rates_abs = list(map(abs, devi_rates))
            min_abs = min(devi_rates_abs)
            if min_abs in devi_rates:  # when min is +
                min_devi_rate = min_abs
            else:  # when min is -
                min_devi_rate = -min_abs
            i_min = devi_rates.index(min_devi_rate)
            simi_input = ref.iloc[indexes[i_min], :4]
            result = ref.iloc[indexes[i_min], 4:]
            # print(indexes)
            # print(devi_rates)
            return [indexes[i_min], min_devi_rate, simi_input, result]
        else:  # when no result found
            print('Result meeting the requirement not found.')
            return [None, None, None, None]


In [257]:
print('Original data for testing:\n', df_test.iloc[DAY - df_test.index[0],])

Original data for testing:
 Temperature(C)                          10.10
Humidity(percent)                       60.00
Radiation(MJ/m2/d)                      21.85
CO2(ppm)                               300.00
Photosynthesis rate(kg C/ha/day)         6.67
Root respiration rate(kg C/ha/day)       1.39
LAI                                      0.11
Shoot respiration rate(kg C/ha/day)      0.70
Name: 120, dtype: float64


In [258]:
# Call find_similar to find the closest result from DNDC data
result1 = find_similar(df_test, df)
print('Result:', result1)

Result: [41, 0.1560524296634302, Temperature(C)          8.45
Humidity(percent)      60.00
Radiation(MJ/m2/d)     21.43
CO2(ppm)              300.00
Name: 101, dtype: float64, Photosynthesis rate(kg C/ha/day)       7.68
Root respiration rate(kg C/ha/day)     1.81
LAI                                    0.10
Shoot respiration rate(kg C/ha/day)    0.66
Name: 101, dtype: float64]


In [259]:
# a function to do min-max normalization of df_test, column by column, using the scalers fitted with df
def inverse_norm(data_norm):
    data = data_norm.copy()
    for column in df.columns.tolist():
        scaler1.fit(df[column].values.reshape(-1,1))  # fit to train data
        data[column] = scaler1.inverse_transform(data_norm[column].values.reshape(-1,1))  # inverse_transform data
    return data


# a function to generate the sequenced data for the GRU model
def load_data_predict(input):
    if type(input) is not np.ndarray:
        data_raw = input.to_numpy() # convert df to numpy array
    else:
        data_raw = input
    data = []
    # create all sequences of length(seq_len-1)
    for i in range(len(data_raw) - seq_len + 1):
        data.append(data_raw[i: i + seq_len - 1])
    data = np.array(data)  # convert list to numpy array
    return data


In [260]:
# A while loop to generate more data using GRU model and search for more results
results = []
round = 0
new_data = df_norm.copy()  # data already normalized
i_result = result1[0]
while round < ROUND_MAX:
    # add a random factor to the original data each round:
    new_data = new_data.mul(random.uniform(0.9,1.1))
    # another way:
    # noise = np.random.normal(loc=0, scale=1, size=new_data.shape)
    # new_data = new_data + noise

    # select the suitable data section for prediction
    if (i_result-BACK) >= seq_len:  # i in the optimal range
        to_pred = new_data.iloc[: i_result-BACK, :]
    elif (seq_len+BACK) > i_result > seq_len:  # i too small, skip BACK
        to_pred = new_data.iloc[: i_result, :]
    elif i_result <= seq_len:  # i way too small, use the minimal length -> seq_len
        print(f'Warning: Non-sufficient data for GRU prediction (require data length >= seq_len) in round {round+1}. Search continues with adjusted data.')
        to_pred = new_data.iloc[: seq_len, :]
    else:  # this should not happen since i within range(len(df))
        raise ValueError("Index out of range of data frame.")

    # A loop to keep predicting until length reaching FORWARD
    preds = to_pred.copy().iloc[-seq_len:, :].to_numpy()  # we need seq_len number of rows for make the next prediction
    while (len(preds)) < FORWARD:
        to_pred_processed = load_data_predict(preds[-seq_len:])
        pred = model_GRU.predict(to_pred_processed)
        preds = np.concatenate((preds, pred))

    # convert to df and reverse min-max normalization to get original data
    preds_df = pd.DataFrame(data=preds, columns=df.columns, dtype='float64')
    preds_ori = inverse_norm(preds_df)

    # call find similar function and append to results
    result_new = find_similar(df_test, preds_ori)
    results.append(result_new)
    i_result = result_new[0]
    if i_result is None:  # if result not found in this round
        i_result = result1[0]  # use the initial index to re-search
        print('Result not found in this round. Search continues with adjusted data.')
    round += 1
    print('round:',round)

流式输出内容被截断，只能显示最后 5000 行内容。
1/1 [==============================] - 0s 24ms/step
round: 29
1/1 [==============================] - 0s 26ms/step
round: 30
1/1 [==============================] - 0s 27ms/step
round: 31
1/1 [==============================] - 0s 25ms/step
round: 32
1/1 [==============================] - 0s 30ms/step
round: 33
1/1 [==============================] - 0s 41ms/step
round: 34
1/1 [==============================] - 0s 23ms/step
round: 35
1/1 [==============================] - 0s 35ms/step
round: 36
1/1 [==============================] - 0s 25ms/step
round: 37
1/1 [==============================] - 0s 40ms/step
round: 38
1/1 [==============================] - 0s 29ms/step
round: 39
1/1 [==============================] - 0s 25ms/step
round: 40
1/1 [==============================] - 0s 40ms/step
round: 41
1/1 [==============================] - 0s 33ms/step
round: 42
1/1 [==============================] - 0s 49ms/step
round: 43
1/1 [==============================] - 0s 3

In [261]:
# find the most close one from the results of all the rounds
from operator import itemgetter
results_exist = [result for result in results if result[0] is not None]  # exclude the None results
result_best = min(results_exist,key=itemgetter(1))
print(result_best)

[7, 0.2062126877148654, Temperature(C)          8.900625
Humidity(percent)      60.000000
Radiation(MJ/m2/d)     19.888127
CO2(ppm)              300.000000
Name: 7, dtype: float64, Photosynthesis rate(kg C/ha/day)       8.309631
Root respiration rate(kg C/ha/day)     2.066644
LAI                                    0.096874
Shoot respiration rate(kg C/ha/day)    0.645826
Name: 7, dtype: float64]


In [262]:
# 3. calculate the errors of the result
result_true = df_test.iloc[DAY - df_test.index[0], 4:]

dndc_errs = [(result_true[n] - result1[-1][n]) / result_true[n] for n in range(len(result_true))]
dndc_err = sum(list(map(abs, dndc_errs))) / len(dndc_errs)
print('DNDC crop error: ', dndc_err)

errs = [(result_true[n] - result_best[-1][n]) / result_true[n] for n in range(len(result_true))]
final_err = sum(list(map(abs, errs))) / len(errs)
print('GRU crop error: ', final_err)

DNDC crop error:  0.15040862732232874
GRU crop error:  0.23233373639895596


# Step 5: Use DNN model to predict the 24-hour crop growth

*   Use DNN model to predict the 24-hour crop growth from the closest environment parameters found within GRU generation in Step 3

In [263]:
# This is just a demo code using the current example data.
# Greenlight and DNDC data are incoherent, revision needed for real input.
input = pd.DataFrame(data=result_best[2]).transpose()
input = input[['CO2(ppm)','Temperature(C)','Humidity(percent)']]

In [264]:
# Normalize
input_norm = input.copy()  # fit to DNN data and transform the result
for column in input.columns.tolist():
    scaler2.fit(df_aug[column].values.reshape(-1,1))
    input_norm[column] = scaler2.transform(input_norm[column].values.reshape(-1,1))
# Predict
hourly_norm = model_DNN.predict(input_norm)

1/1 [==============================] - 0s 26ms/step


In [265]:
# convert to df and reverse normalization to get original data
hourly_df_norm = pd.DataFrame(data=hourly_norm, columns=['0', '1', '2', '3',
       '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21', '22', '23'], dtype='float64')
hourly_df = hourly_df_norm.copy()  # fit to DNN data and back transform the result
for column in hourly_df.columns.tolist():
    scaler2.fit(df_aug[column].values.reshape(-1,1))
    hourly_df[column] = scaler2.inverse_transform(hourly_df[column].values.reshape(-1,1))

In [266]:
print(hourly_df)

              0             1            2             3             4  \
0  36757.246374  36504.877498  37332.53286  36705.987978  36527.208136   

              5             6             7             8             9  ...  \
0  35841.263099  37361.324793  37715.320787  38084.903407  38877.750912  ...   

           14            15            16           17            18  \
0  42120.7419  42389.374194  43775.090022  44279.43722  44344.932592   

             19            20           21            22            23  
0  44639.292763  45201.902813  45835.06099  45762.517177  46211.727479  

[1 rows x 24 columns]


In [267]:
print(hourly_df.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'],
      dtype='object')


# Calibrate and Get the final crop growth result
*   Calibrate the output data from DNN to day input of Step 1. Calculate the difference rate, and multiply the rate to the value of each hour value.
*   Get the final crop growth result of the specified hour from the calibrated result (Select the hour from step 1)


In [268]:
# calculate delta day
delta_d = hourly_df.at[0, '23'] - hourly_df.at[0, '0']
# calculate delta hour
delta_h = hourly_df - hourly_df.shift(axis=1)

In [269]:
print(delta_d)
print(delta_h)

9454.481105637256
    0           1           2           3           4           5  \
0 NaN -252.368876  827.655363 -626.544882 -178.779842 -685.945037   

             6           7          8           9  ...           14  \
0  1520.061694  353.995994  369.58262  792.847505  ...  1554.316347   

           15           16          17         18          19         20  \
0  268.632294  1385.715828  504.347199  65.495372  294.360171  562.61005   

           21         22          23  
0  633.158177 -72.543813  449.210303  

[1 rows x 24 columns]


In [270]:
# get the 4 crop features predicted by GRU
crop = pd.DataFrame(data=result_best[3]).transpose()
print(crop)

   Photosynthesis rate(kg C/ha/day)  Root respiration rate(kg C/ha/day)  \
7                          8.309631                            2.066644   

        LAI  Shoot respiration rate(kg C/ha/day)  
7  0.096874                             0.645826  


In [271]:
# scale the 4 hourly crop features according to daily sum
crop_photo = delta_h / (delta_d / crop.at[crop.index[0], 'Photosynthesis rate(kg C/ha/day)'])
crop_root = delta_h / (delta_d / crop.at[crop.index[0], 'Root respiration rate(kg C/ha/day)'])
crop_shoot = delta_h / (delta_d / crop.at[crop.index[0], 'Shoot respiration rate(kg C/ha/day)'])
crop_lai = delta_h / (delta_d / crop.at[crop.index[0], 'LAI'])

In [272]:
print(crop_photo)

    0         1         2         3         4         5         6        7  \
0 NaN -0.221809  0.727434 -0.550676 -0.157131 -0.602883  1.335996  0.31113   

         8         9  ...        14        15        16        17        18  \
0  0.32483  0.696841  ...  1.366103  0.236103  1.217919  0.443275  0.057564   

         19        20        21        22        23  
0  0.258716  0.494483  0.556489 -0.063759  0.394815  

[1 rows x 24 columns]


In [273]:
# Get the final result at specified hour
# Note: For the test this step is not applicable. Below is just for a demonstration.
hour = '12'
result_h = {'Photosynthesis rate(kg C/ha/day)': crop_photo.at[0, hour],
            'Root respiration rate(kg C/ha/day)': crop_root.at[0, hour],
            'Shoot respiration rate(kg C/ha/day)': crop_shoot.at[0, hour],
            'LAI': crop_lai.at[0, hour]}

print(f'The predicted crop growth at hour {hour}:\n', result_h)

The predicted crop growth at hour 12:
 {'Photosynthesis rate(kg C/ha/day)': 0.19871726795931105, 'Root respiration rate(kg C/ha/day)': 0.049421911202315684, 'Shoot respiration rate(kg C/ha/day)': 0.015444347250723653, 'LAI': 0.002316652087608548}
